In [30]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)

In [31]:
trades_path = "../data/raw/historical_data.csv"
sentiment_path = "../data/raw/fear_greed_index.csv"

df_trades = pd.read_csv(trades_path)
df_sentiment = pd.read_csv(sentiment_path)

In [32]:
def audit_dataframe(df, name):
    print(f"\n{'='*60}")
    print(f"DATASET: {name}")
    print(f"{'='*60}")

    print(f"Rows: {df.shape[0]}")
    print(f"Columns: {df.shape[1]}\n")

    print("Column Types:")
    print(df.dtypes)

    print("\nMissing Values:")
    print(df.isna().sum())

    print("\nDuplicate Rows:", df.duplicated().sum())

    print("\nSample Data:")
    display(df.head())

In [33]:
audit_dataframe(df_trades, "Historical Trader Data")
audit_dataframe(df_sentiment, "Fear & Greed Sentiment Data")


DATASET: Historical Trader Data
Rows: 211224
Columns: 16

Column Types:
Account              object
Coin                 object
Execution Price     float64
Size Tokens         float64
Size USD            float64
Side                 object
Timestamp IST        object
Start Position      float64
Direction            object
Closed PnL          float64
Transaction Hash     object
Order ID              int64
Crossed                bool
Fee                 float64
Trade ID            float64
Timestamp           float64
dtype: object

Missing Values:
Account             0
Coin                0
Execution Price     0
Size Tokens         0
Size USD            0
Side                0
Timestamp IST       0
Start Position      0
Direction           0
Closed PnL          0
Transaction Hash    0
Order ID            0
Crossed             0
Fee                 0
Trade ID            0
Timestamp           0
dtype: int64

Duplicate Rows: 0

Sample Data:


,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,Transaction Hash,Order ID,Crossed,Fee,Trade ID,Timestamp
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,02-12-2024 22:50,0.000000,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.345404,8.950000e+14,1.730000e+12
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,BUY,02-12-2024 22:50,986.524596,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.005600,4.430000e+14,1.730000e+12
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,BUY,02-12-2024 22:50,1002.518996,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050431,6.600000e+14,1.730000e+12
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,BUY,02-12-2024 22:50,1146.558564,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050043,1.080000e+15,1.730000e+12
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,BUY,02-12-2024 22:50,1289.488521,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.003055,1.050000e+15,1.730000e+12



DATASET: Fear & Greed Sentiment Data
Rows: 2644
Columns: 4

Column Types:
timestamp          int64
value              int64
classification    object
date              object
dtype: object

Missing Values:
timestamp         0
value             0
classification    0
date              0
dtype: int64

Duplicate Rows: 0

Sample Data:


,timestamp,value,classification,date
0,1517463000,30,Fear,2018-02-01
1,1517549400,15,Extreme Fear,2018-02-02
2,1517635800,40,Fear,2018-02-03
3,1517722200,24,Extreme Fear,2018-02-04
4,1517808600,11,Extreme Fear,2018-02-05


In [34]:
df_trades['timestamp_ist'] = pd.to_datetime(
    df_trades['Timestamp IST'],
    format="%d-%m-%Y %H:%M"
)

df_trades['date'] = df_trades['timestamp_ist'].dt.date

In [35]:
df_sentiment['timestamp'] = pd.to_datetime(
    df_sentiment['timestamp'],
    unit='s'
)

df_sentiment['date'] = df_sentiment['timestamp'].dt.date

In [36]:
df_sentiment_clean = df_sentiment[['date', 'classification', 'value']]

In [37]:
df_merged = df_trades.merge(
    df_sentiment_clean,
    on='date',
    how='left'
)

In [38]:
total_rows = len(df_merged)
missing_sentiment = df_merged['classification'].isna().sum()

print(f"Total trades: {total_rows}")
print(f"Trades without sentiment: {missing_sentiment}")
print(f"Coverage: {(1 - missing_sentiment/total_rows)*100:.2f}%")

Total trades: 211224
Trades without sentiment: 6
Coverage: 100.00%


In [39]:
df_merged.to_csv("../data/intermediate/merged_daily_base.csv", index=False)

In [40]:
df = pd.read_csv("../data/intermediate/merged_daily_base.csv")

In [41]:
df.columns = df.columns.str.lower().str.replace(" ", "_")

In [42]:
df['leverage_proxy'] = df['size_usd'] / df['execution_price']

In [43]:
trader_daily = df.groupby(['account', 'date']).agg(
    daily_pnl=('closed_pnl', 'sum'),
    total_volume=('size_usd', 'sum'),
    avg_trade_size=('size_usd', 'mean'),
    trade_count=('size_usd', 'count'),
    avg_leverage=('leverage_proxy', 'mean'),
    pnl_volatility=('closed_pnl', 'std')
).reset_index()

In [44]:
df['win'] = (df['closed_pnl'] > 0).astype(int)

win_rate = df.groupby(['account', 'date'])['win'].mean().reset_index(name='win_rate')

trader_daily = trader_daily.merge(win_rate, on=['account', 'date'])

In [45]:
trader_daily['risk_adjusted_return'] = (
    trader_daily['daily_pnl'] / (trader_daily['pnl_volatility'] + 1e-6)
)

In [46]:
market_daily = df.groupby('date').agg(
    total_trades=('account', 'count'),
    total_pnl=('closed_pnl', 'sum'),
    avg_leverage=('leverage_proxy', 'mean'),
    pnl_volatility=('closed_pnl', 'std'),
    active_traders=('account', 'nunique')
).reset_index()

In [47]:
market_daily['avg_leverage'] = df.groupby('date')['leverage_proxy'].mean()

In [48]:
direction_counts = df.pivot_table(
    index='date',
    columns='direction',
    values='account',
    aggfunc='count',
    fill_value=0
).reset_index()

direction_counts['long_short_ratio'] = (
    direction_counts.get('Buy', 0) /
    (direction_counts.get('Sell', 0) + 1)
)

In [49]:
trader_daily['aggression_score'] = (
    trader_daily['trade_count'] * trader_daily['avg_leverage']
)

In [50]:
trader_daily['capital_efficiency'] = (
    trader_daily['daily_pnl'] / (trader_daily['total_volume'] + 1e-6)
)

In [51]:
trader_daily['consistency_index'] = (
    trader_daily['win_rate'] / (trader_daily['pnl_volatility'] + 1e-6)
)

In [52]:
trader_daily.to_csv("../data/processed/trader_daily_metrics.csv", index=False)
market_daily.to_csv("../data/processed/market_daily_metrics.csv", index=False)
direction_counts.to_csv("../data/processed/long_short_daily.csv", index=False)